## Let us do some tests related to the efficiency of numpy and scipy functions

In [1]:
import numpy as np

import scipy.sparse

from scipy.sparse import csc_matrix, csr_matrix

### Generate sparse random data

In [2]:
n = 5000
dim = 2000
A = csr_matrix(scipy.sparse.random(n, dim))
x = csr_matrix(scipy.sparse.random(dim, 1))
A_ = csc_matrix(A)
x_ = csc_matrix(x)

### Compare csc_matrix and csr_matrix

In [3]:
%timeit A.dot(x)
%timeit A.dot(x_)
%timeit A_.dot(x)
%timeit A_.dot(x_)

318 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
369 µs ± 6.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
151 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
99 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Let's check that the results match.

In [4]:
abs(A.dot(x) - A.dot(x_)).sum()

0.0

#### Is Numpy faster?

In [5]:
x = x.toarray().squeeze()
A = A.toarray()

In [6]:
%timeit A @ x

4.23 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Now let us sample rows (used in SGD)

In [7]:
def fake_stochastic_gradient(A, x):
    n = A.shape[0]
    i = np.random.choice(n)
    return A[i].dot(x)

In [8]:
%timeit fake_stochastic_gradient(A, x)
%timeit fake_stochastic_gradient(A, x_)
%timeit fake_stochastic_gradient(A_, x)
%timeit fake_stochastic_gradient(A_, x_)

5.74 µs ± 56.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
72.2 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
229 µs ± 8.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
322 µs ± 9.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Conclusions:
### 1. Use csc for deterministic (full batch) gradient computation
### 2. Use csr if stochastic gradients are required

## Efficiencty of other sparse-vector operations

#### Sparse vector

In [9]:
x = csr_matrix([1] + list(np.zeros(1000000)))
x

<1x1000001 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [10]:
%timeit x + x
%timeit x * 2
%timeit abs(x)
%timeit x.minimum(0.5)
%timeit x.eliminate_zeros()

77 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
39.2 µs ± 2.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
35.9 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
38.1 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
4.41 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Dense vector

In [11]:
x = csr_matrix(np.arange(1000000))
x

<1x1000000 sparse matrix of type '<class 'numpy.int64'>'
	with 999999 stored elements in Compressed Sparse Row format>

In [12]:
%timeit x + x
%timeit x * 2
%timeit abs(x)
%timeit x.minimum(0.5)
%timeit x.eliminate_zeros()

4.27 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.9 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.81 ms ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.52 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
680 µs ± 39.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Random number generation

### Todo